# Enhanced NLP Pipeline for Global Question Extraction (Bis)

This notebook builds on our previous NLP workflows to produce a refined, multilingual mapping of individual survey items to overarching “global” themes. We:

1. **Set up and load data**  
   - Import required NLP, data-wrangling, and clustering libraries  
   - Load the master list of global questions and all sheets from the CodeBook  

2. **Clean and merge question metadata**  
   - Apply year-specific filters to legacy vs. recent survey questions  
   - Normalize `num_question` codes and drop empty entries  
   - Concatenate all sheets into one unified DataFrame  

3. **Preprocess text for NLP**  
   - Lowercase, tokenize, remove stop-words and punctuation  
   - Lemmatize tokens to standardize word forms  

4. **Compute semantic similarities**  
   - Generate embeddings using both BERT and DSBSAN transformer models  
   - Vectorize labels with TF-IDF and compute cosine similarity matrices  
   - Compare transformer vs. TF-IDF approaches to choose the optimal similarity metric  

5. **Cluster into global themes**  
   - Use DBSCAN on similarity distances to group related questions  
   - Select the top global themes based on a similarity threshold  

6. **Extract and compile global question table**  
   - Build a final DataFrame mapping each question code back to its global parent  
   - Transpose and align label options for consistency  

7. **Translate global labels**  
   - Attempt LibreTranslate API (not available locally)  
   - Fallback to BERT-based translation models for high-quality English labels  
   - Note unsupported languages (e.g., Romanche) where translations aren’t available  

8. **Export final results**  
   - Save the enriched, multilingual `global_questions_nlp_bis.csv` for downstream analysis and reporting  


# Import

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
from tqdm import tqdm
from transformers import BertTokenizer, BertModel, AutoTokenizer, AutoModel
import torch
import os
import re
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from nltk.corpus import stopwords
import spacy
from nltk.stem import WordNetLemmatizer
import requests
import time
from transformers import pipeline
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, MarianMTModel, MarianTokenizer


# File loading

In [ ]:
# Load the files 
question_globales_path = 'data/QuestionGlobales.xlsx'
extraction_codebook_path = 'data/Extraction CodeBook - 3. Cleaned.xlsx'
gsb_path = 'data/GSB 2023_V1.xlsx'

# Read the global questions file
df_globales = pd.read_excel(question_globales_path)

# Read all sheets from the codebook file
sheets_codebook = pd.read_excel(extraction_codebook_path, sheet_name=None)

# answers dataframe
gsb_df = pd.read_excel(gsb_path) 


In [ ]:
gsb_df.head()

## Start of data manipulation before NLP preprocessing pipeline

In [ ]:
# create a dataframe for each year
years = ['1988', '1994', '1998', '2005', '2009', '2017', '2023']
dataframes = {}

# choose the columns to keep
columns_to_keep = ['code', 'num_question', 'year', 'label', 'type', 'format']


for year in years:
    if year in sheets_codebook:  # ensure the sheet exists
        # each sheet is a DataFrame
        df = sheets_codebook[year]
        # keep only the columns we need (defined in columns_to_keep)
        dataframes[f'main_questions_{year}'] = df[columns_to_keep]
    else:
        print(f"Sheet for year {year} not found.")

# define the dataframes
main_questions_1988 = dataframes.get('main_questions_1988', pd.DataFrame())
main_questions_1994 = dataframes.get('main_questions_1994', pd.DataFrame())
main_questions_1998 = dataframes.get('main_questions_1998', pd.DataFrame())
main_questions_2005 = dataframes.get('main_questions_2005', pd.DataFrame())
main_questions_2009 = dataframes.get('main_questions_2009', pd.DataFrame())
main_questions_2017 = dataframes.get('main_questions_2017', pd.DataFrame())
main_questions_2023 = dataframes.get('main_questions_2023', pd.DataFrame())

main_questions_2023.head()

## Sheet filtering for num_question

In [ ]:
# tailor made data cleaning for the old years sheets 
def filter_num_question_old(df):
    # delete rows with empty num_question
    df = df[df['num_question'].notna()].copy()  

    # convert num_question to string
    df.loc[:, 'num_question'] = df['num_question'].astype(str)  

    # some tailor made filtering --> keep only rows with num_question that are digits or contain 'a' or 'A' 
    df = df[df['num_question'].str.match(r'^\d+$|.*[aA].*')]
    
    # keep only the first occurence of each num_question
    df = df.drop_duplicates(subset='num_question', keep='first')

    # delete the a or A from the num_question
    df['num_question'] = df['num_question'].str.replace('a', '', regex=False).str.replace('A', '', regex=False)

    # keep only the first occurence of each num_question
    df = df.drop_duplicates(subset='num_question', keep='first')

    return df

In [ ]:
# tailor made data cleaning for the new years sheets 
def filter_num_question_new(df):
    # delete rows with empty num_question
    df = df[df['num_question'].notna()].copy()  
    # keep only the first occurence of each num_question
    df = df.drop_duplicates(subset='num_question', keep='first')
    return df

In [ ]:
main_questions_1988 = filter_num_question_old(main_questions_1988)
main_questions_1994 = filter_num_question_old(main_questions_1994)
main_questions_1998 = filter_num_question_old(main_questions_1998)
main_questions_2005 = filter_num_question_old(main_questions_2005)
main_questions_2009 = filter_num_question_old(main_questions_2009)
main_questions_2017 = filter_num_question_new(main_questions_2017)
main_questions_2017 = main_questions_2017[main_questions_2017['num_question'] != 'Q1']
main_questions_2023 = filter_num_question_new(main_questions_2023)


In [ ]:
main_questions_2017.head()

In [ ]:
dataframes_to_merge = [
    main_questions_1988,
    main_questions_1994,
    main_questions_1998,
    main_questions_2005,
    main_questions_2009,
    main_questions_2017,
    main_questions_2023
]

# merge all the dataframes
merged_df = pd.concat(dataframes_to_merge, ignore_index=True)

In [ ]:
merged_df.head()

## Start of text preprocessing

In [ ]:
!python -m spacy download de_core_news_sm

In [ ]:
nlp = spacy.load('de_core_news_sm')

def preprocess_text(text):
    # 1. convert text to lowercase
    text = text.lower()
    
    # 2. delete punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # 3. delete extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    # 4. lemmatization + tokenization + remove stopwords
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop] 
    
    return tokens  

# apply the preprocess_text function to the 'label' column
tqdm.pandas() 
merged_df['tokens'] = merged_df['label'].progress_apply(preprocess_text)


In [ ]:
merged_df.head()

In [ ]:
merged_df[merged_df['year'] == 2023]

# NLP pipeline: 

## Transformer version: Bert

In [ ]:
model_name = 'dbmdz/bert-base-german-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [ ]:
def get_embeddings(labels):
    embeddings = []
    for label in tqdm(labels, desc='Generating embeddings'):
        inputs = tokenizer(label, return_tensors='pt', padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            embedding = outputs.last_hidden_state[:, 0, :].numpy()
            embeddings.append(embedding)
    return np.vstack(embeddings)


In [ ]:
# obtain the tokens from the merged DataFrame
labels = merged_df['tokens'].apply(lambda x: ' '.join(x)).unique()  # convert the list of tokens to a string

# obtain the embeddings for the tokens
embeddings = get_embeddings(labels)

# use cosine similarity to calculate the similarity matrix
similarity_matrix = cosine_similarity(embeddings)

# create a DataFrame from the similarity matrix
similarity_df = pd.DataFrame(similarity_matrix, index=labels, columns=labels)

# set the threshold for similarity --> here 0.97 but can be changed
threshold = 0.97

similar_pairs = []

# iterate over the similarity matrix to identify similar pairs
for i in tqdm(range(len(labels)), desc='Identifying similar pairs'):
    for j in range(i + 1, len(labels)):  
        if similarity_matrix[i][j] >= threshold:
            similar_pairs.append((labels[i], labels[j]))

# create a new DataFrame to store the combined rows
question_globale_new = pd.DataFrame(columns=['id', 'label', 'year', 'code'])

id_counter = 1

for label1, label2 in similar_pairs:
    # add a row for each pair of similar tokens
    temp_df = merged_df[(merged_df['tokens'].apply(lambda x: ' '.join(x)) == label1) | 
                        (merged_df['tokens'].apply(lambda x: ' '.join(x)) == label2)]
    combined_row = {
        'id': id_counter,  # ad id for each combined row
        'label': f"{label1}; {label2}",
        'year': '; '.join(temp_df['year'].astype(str)),
        'code': '; '.join(temp_df['code'])
    }

    question_globale_new = pd.concat([question_globale_new, pd.DataFrame([combined_row])], ignore_index=True)

    id_counter += 1


In [ ]:
question_globale_new.head()

In [ ]:
question_globale_new.shape

In [ ]:
question_globale_new[question_globale_new['id'] == 1].label.values[0]

## Transformer version: DSBSAN

In [ ]:
# obtain the tokens from the merged DataFrame
labels = merged_df['tokens'].apply(lambda x: ' '.join(x)).unique()

# embedding model
def get_embeddings(labels):
    embeddings = []
    for label in tqdm(labels, desc="embedding computation", unit="label"):
        inputs = tokenizer(label, return_tensors='pt', padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            embedding = outputs.last_hidden_state[:, 0, :].numpy()  
            embeddings.append(embedding)
    return np.vstack(embeddings)

# obtain unique labels
labels = merged_df['label'].unique()

# obtain embeddings for the labels
embeddings = get_embeddings(labels)
print("Shape of embeddings:", embeddings.shape)

# use cosine similarity to calculate the similarity matrix
distance_matrix = cosine_distances(embeddings)

# apply DBSCAN clustering
dbscan = DBSCAN(eps=0.05, min_samples=3, metric='precomputed')  # use precomputed to pass the distance matrix
clusters = dbscan.fit_predict(distance_matrix)

# create a dictionary to map labels to indices
label_to_index = {label: index for index, label in enumerate(merged_df['label'].unique())}

# create a Series to store the clusters
cluster_series = pd.Series(-1, index=merged_df.index)

# assign the clusters to the labels
for label, cluster_id in zip(labels, clusters):
    index = label_to_index[label]
    cluster_series.iloc[index] = cluster_id

# add the clusters to the merged DataFrame
merged_df['cluster'] = cluster_series

# create a new DataFrame to store the combined rows
question_globale_new = pd.DataFrame(columns=['id', 'label', 'year', 'code'])

# fill the new DataFrame with the combined rows
id_counter = 1
for cluster_id in set(merged_df['cluster']):
    if cluster_id != -1:  
        temp_df = merged_df[merged_df['cluster'] == cluster_id]
        if not temp_df.empty:
            combined_row = {
                'id': id_counter,
                'label': '; '.join(temp_df['label']),
                'year': '; '.join(temp_df['year'].astype(str)),
                'code': '; '.join(temp_df['code'])
            }
            question_globale_new = pd.concat([question_globale_new, pd.DataFrame([combined_row])], ignore_index=True)
            id_counter += 1


In [ ]:
question_globale_new.shape

In [ ]:
question_globale_new.head()

In [ ]:
question_globale_new[question_globale_new['id'] == 1].label.values[0]

## TF-IDF version

### Top 10 global questions

In [ ]:
def has_duplicates(years):
    return len(years) != len(set(years))

# obtain the tokens from the merged DataFrame
labels = merged_df['tokens'].apply(lambda x: ' '.join(x)).unique()

# create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(labels)

# calculate the cosine similarity matrix
similarity_matrix = cosine_similarity(tfidf_matrix)

similarity_matrix[np.tril_indices_from(similarity_matrix)] = -1

top_10_indices = np.dstack(np.unravel_index(np.argsort(similarity_matrix.ravel())[-10:], similarity_matrix.shape))[0]

question_globale_new_top_10 = pd.DataFrame(columns=['id', 'label', 'year', 'code'])

id_counter = 1

# Traiter les 10 paires ayant la plus grande similarité
for i, j in top_10_indices:
    label1 = labels[i]
    label2 = labels[j]
    
    # Ajouter une ligne pour chaque paire de tokens similaires
    temp_df = merged_df[(merged_df['tokens'].apply(lambda x: ' '.join(x)) == label1) | 
                        (merged_df['tokens'].apply(lambda x: ' '.join(x)) == label2)]
    
    combined_years = '; '.join(temp_df['year'].astype(str)).split('; ')

    # Vérifier s'il y a des années dupliquées
    if has_duplicates(combined_years):
        continue  # Ignorer la ligne si des années dupliquées sont trouvées
    
    # Construire la ligne combinée
    combined_row = {
        'id': id_counter,  
        'label': f"{label1}; {label2}",
        'year': '; '.join(combined_years),
        'code': '; '.join(temp_df['code'])
    }

    # Ajouter la ligne au DataFrame question_globale_new
    question_globale_new_top_10 = pd.concat([question_globale_new_top_10, pd.DataFrame([combined_row])], ignore_index=True)

    id_counter += 1


In [ ]:
question_globale_new_top_10.shape

In [ ]:
question_globale_new_top_10.head(50)

In [ ]:
question_globale_new_top_10.loc[question_globale_new_top_10['year'].str.contains('2023'), 'label']


In [ ]:
question_globale_new_top_10[question_globale_new_top_10['id'] == 1].label.values[0]

### Question global selection based on threshold 

In [ ]:
def has_duplicates(years):
    return len(years) != len(set(years))

# obtain the tokens from the merged DataFrame
labels = merged_df['tokens'].apply(lambda x: ' '.join(x)).unique()

# create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(labels)

# calculate the cosine similarity matrix
similarity_matrix = cosine_similarity(tfidf_matrix)

# create a DataFrame from the similarity matrix
similarity_df = pd.DataFrame(similarity_matrix, index=labels, columns=labels)

# define the threshold for similarity, here 0.8 seems to be a good value but can be changed 
threshold = 0.6

similar_pairs = []

# identify similar pairs
for i in tqdm(range(len(labels)), desc='Identifying similar pairs'):
    for j in range(i + 1, len(labels)):
        if similarity_matrix[i][j] >= threshold:
            similar_pairs.append((labels[i], labels[j]))

# create a new DataFrame to store the combined rows
question_globale_new_threshold = pd.DataFrame(columns=['id', 'label', 'year', 'code'])

id_counter = 1

for label1, label2 in similar_pairs:
    # add a row for each pair of similar tokens
    temp_df = merged_df[(merged_df['tokens'].apply(lambda x: ' '.join(x)) == label1) | 
                        (merged_df['tokens'].apply(lambda x: ' '.join(x)) == label2)]
    
    combined_years = '; '.join(temp_df['year'].astype(str)).split('; ')

    if has_duplicates(combined_years):
        continue
    
    combined_row = {
        'id': id_counter,  
        'label': f"{label1}; {label2}",
        'year': '; '.join(combined_years),
        'code': '; '.join(temp_df['code'])
    }

    question_globale_new_threshold = pd.concat([question_globale_new_threshold, pd.DataFrame([combined_row])], ignore_index=True)

    id_counter += 1

In [ ]:
question_globale_new_threshold.shape

### Here the results seems better --> use the TF-IDF version 

# CSV extraction of the global questions

In [ ]:
quest_glob_columns = [
    'label', 'code_first_question','code_other_question','text_de', 'text_fr', 'text_it', 'text_ro', 'text_en',
    'category_label', 'category_text_de', 'category_text_fr',
    'category_text_it', 'category_text_ro', 'category_text_en',
    'options_value', 'options_label'
]

code_to_token = {}

for index, row in merged_df.iterrows():
    code = row['code']
    tokens = row['tokens']

    for token in tokens:
        if not token.isdigit():  
            if code not in code_to_token:  
                code_to_token[code] = token
            break


print(code_to_token)

In [ ]:
question_globale_new_top_10.head()

In [ ]:
# create a new DataFrame to store the final results
df_fin_top_10 = pd.DataFrame(columns=quest_glob_columns)

# fill the new DataFrame with the combined rows
for index, row in question_globale_new_top_10.iterrows():
    codes_list = row['code'].split('; ')
    gsb23_code = next((code for code in row['code'].split('; ') if code.startswith('GSB23_')), None)
    code_first_question = gsb23_code if gsb23_code is not None else row['code'].split('; ')[0]  # use 'GSB23_' code or the first code
    code_other_question = '; '.join([code for code in codes_list if code != code_first_question])
    first_token = code_to_token.get(code_first_question, '')  # use the token corresponding to the first code

    # Find the corresponding text_de in merged_df using code_first_question
    label_from_merged = merged_df.loc[merged_df['code'] == code_first_question, 'label'].values
    if len(label_from_merged) > 0:
        # Remove any leading numbers and dots (e.g., "52. " or "123. ")
        cleaned_label = re.sub(r'^\d+\.\s*', '', label_from_merged[0])
        text_de_first_question = cleaned_label
    else:
        text_de_first_question = row['label'].split('; ')[0]

    # create a new row with the required columns
    new_row = {
        'label': first_token,  # use the first code in 'label'
        'code_first_question': code_first_question,  # put the first code in 'code_first_question'
        'code_other_question': code_other_question,  # put the other codes in 'code_other_question'
        'text_de': text_de_first_question,  # use the first label in 'text_de'
        'text_fr': '',  
        'text_it': '', 
        'text_ro': '', 
        'text_en': '',
        'category_label': '',
        'category_text_de': '',
        'category_text_fr': '',
        'category_text_it'  : '',
        'category_text_ro': '',
        'category_text_en': '',
        'options_value': '',
        'options_label': ''
    }

    # add the new row to the final DataFrame
    df_fin_top_10 = pd.concat([df_fin_top_10, pd.DataFrame([new_row])], ignore_index=True)


df_fin_top_10.head()


In [ ]:
gsb_columns = gsb_df.columns

# fill the 'options_value' column in df_fin
for index, row in df_fin_top_10.iterrows():
    # get the code of the first question
    question_code = row['code_first_question']
    
    # check if the question code exists in the 'GSB 2023_V1' DataFrame
    if question_code in gsb_columns:
        # extract the unique values from the column
        unique_values = gsb_df[question_code].dropna().unique()
        
        # if there are unique values, join them 
        if len(unique_values) > 0:
            options_value = ";".join(map(str, unique_values))
        else:
            options_value = ''
        
        # fill the options_value in the final DataFrame
        df_fin_top_10.at[index, 'options_value'] = options_value
    else:
        # if the question code does not exist in the 'GSB 2023_V1' DataFrame, fill an empty string
        # to be adapted when other version of the answers than 2023 will be available
        df_fin_top_10.at[index, 'options_value'] = ''


In [ ]:
df_fin_top_10.head(10)

## Translation of the global question using an open source API


### Libretranslate -> not working locally, instead do : 

### Bert model to translate 

In [ ]:
# model selection, also tried "Helsinki-NLP/opus-mt-de-fr" but the translation was not as good
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

def translate_mbart(text, source_lang, target_lang):
    # source language
    tokenizer.src_lang = source_lang
    # encode
    encoded_input = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
    generated_tokens = model.generate(
        **encoded_input,
        forced_bos_token_id=tokenizer.lang_code_to_id[target_lang],
        max_length=512,
        num_beams=4,  # use beam search to improve the translation
        early_stopping=True
    )
    # decode the generated tokens
    return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]



In [ ]:
df_fin_top_10['text_en'] = df_fin_top_10['text_de'].apply(lambda x: translate_mbart(x, "de_DE", "en_XX"))
df_fin_top_10['text_fr'] = df_fin_top_10['text_en'].apply(lambda x: translate_mbart(x, "en_XX", "fr_XX"))
df_fin_top_10['text_it'] = df_fin_top_10['text_en'].apply(lambda x: translate_mbart(x, "en_XX", "it_IT"))
# romanche not available in the model

In [ ]:
df_fin_top_10['text_ro'] = 'Not available for the moment'

In [ ]:
df_fin_top_10.head(10)

### Results extraction

In [ ]:
df_fin_top_10.to_csv('data/top_10_QuestionGlobales_NLP.csv', index=False, encoding='utf-8')